In [1]:
import torch
from torch import nn
from torchvision.datasets import MNIST
import torch.nn.functional as F

In [ ]:
# Determines if a GPU is available 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.log_softmax(x)

In [3]:
model = Network()

In [4]:
class Pruning(nn.Module):
    def __init__(self, model, strategy_iters, strategy_method=''):
        super(Pruning, self).__init__()
        self.model = model
        self.strategy_iters = range(strategy_iters)
        self.strategy_method = strategy_method
    
    def forward(self, x):
        return self.model(x)
    
    
    

In [5]:
model = Pruning(model, 3)

In [7]:
import sys
sys.path.append("../..")

In [8]:
from data import mnist

In [9]:
train_loader, valid_loader = mnist.get_train_valid_loader(batch_size=16)

In [13]:
from torch import optim

epochs = 3
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Training loop
model.to(device)
#  we do a number of iterations of network pruning and for each one of them separate training
for i in model.strategy_iters: 
    # the regular pytorch training for loop
    model.train()
    for e in range(epochs):
        print('Epoch:', e+1)
        for image, label in train_loader:
            image, label = image.to(device), label.to(device)
            optimizer.zero_grad()
            image = image.view(image.shape[0], -1)
            output = model(image)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            
            
    # when the training for the current network is done, 
    # get the pruned network and do the training for it
    model = model.next_model() # pruned model
    
    # get the lr and other stuff from the current optimizer to the new one
    optimizer_new = optim.Adam(network.parameters())
    optimizer_new.load_state_dict(optimizer.state_dict())
    optimizer = optimizer_new

Epoch: 0
Epoch: 1
Epoch: 2


NameError: name 'network' is not defined

In [16]:
optimizer.state_dict()

{'state': {139780507265856: {'step': 9000,
   'exp_avg': tensor([[ 1.8191e-05,  1.8191e-05,  1.8191e-05,  ...,  1.8191e-05,
             1.8191e-05,  1.8191e-05],
           [-1.6610e-04, -1.6610e-04, -1.6610e-04,  ..., -1.6610e-04,
            -1.6610e-04, -1.6610e-04],
           [ 6.2579e-04,  6.2579e-04,  6.2579e-04,  ...,  6.2579e-04,
             6.2579e-04,  6.2579e-04],
           ...,
           [-5.2239e-04, -5.2239e-04, -5.2239e-04,  ..., -5.2239e-04,
            -5.2239e-04, -5.2239e-04],
           [-2.3842e-04, -2.3842e-04, -2.3842e-04,  ..., -2.3842e-04,
            -2.3842e-04, -2.3842e-04],
           [ 2.4798e-04,  2.4798e-04,  2.4798e-04,  ...,  2.4798e-04,
             2.4798e-04,  2.4798e-04]]),
   'exp_avg_sq': tensor([[4.2071e-07, 4.2071e-07, 4.2071e-07,  ..., 4.2071e-07, 4.2071e-07,
            4.2071e-07],
           [4.7680e-06, 4.7680e-06, 4.7680e-06,  ..., 4.7680e-06, 4.7680e-06,
            4.7680e-06],
           [4.4256e-06, 4.4256e-06, 4.4256e-06,  ..., 